# Deloitte's Quantum Climate Challenge 2023

The challenge focuses on CO2 captuere in MOFs (Metal Organic Frameworks) ans is dvided in two principal tasks

## Task 1: Calculate the minimum of the potential energy surface of combinations gas molecules and metallic ions

### Task 1A: Build a quantum/quantum-hybrid algorithm. Run simulations and on real quantum devices

Pick at least one metallic ion from the list:
* __Mg2+ (2p6 - 10 e-)__
* Mn2+ (3d5 - 23 e-)
* Fe2+ (3d6 - 24 e-)
* Co2+ (3d7 - 25 e-)
* Ni2+ (3d8 - 26 e-)
* Cu2+ (3d9 - 27 e-)
* __Zn2+ (3d10 - 28 e-)__

And study the composite system with CO2 and another gas molecule:
* __CO2 (22 e-)__
* __H2O (10 e-)__
* __N2 (14e-)__


### Task 1B: Compare those results to classical simulations

For the purpose of this notebook we will focus on Task 1A. Here we import the resulta from

### Imports and Helper Functions

In [1]:
from qiskit.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver #Algorithms

#Qiskit odds and ends
from qiskit.circuit.library import EfficientSU2, EvolvedOperatorAnsatz
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP, L_BFGS_B
from qiskit.opflow import Z2Symmetries, X, Y, Z, I, PauliSumOp, Gradient, NaturalGradient
from qiskit import IBMQ, BasicAer, Aer, transpile
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.utils.mitigation import CompleteMeasFitter #Measurement error mitigatioin
from qiskit.tools.visualization import circuit_drawer
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.algorithms.minimum_eigensolvers import VQE, AdaptVQE, MinimumEigensolverResult
from qiskit.primitives import Estimator
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit.quantum_info import SparsePauliOp

#qiskit_nature
from qiskit_nature.second_q.drivers import PySCFDriver, MethodType
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.circuit.library import UCCSD, PUCCD, SUCCD, HartreeFock, CHC, VSCF
from qiskit_nature.second_q.operators.fermionic_op import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer , FreezeCoreTransformer
from qiskit_nature.second_q.problems import ElectronicStructureProblem, EigenstateResult
from qiskit_nature.second_q.mappers import QubitConverter, ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.second_q.algorithms.ground_state_solvers.minimum_eigensolver_factories.vqe_ucc_factory import VQEUCCFactory
from qiskit_nature.second_q.algorithms.ground_state_solvers.minimum_eigensolver_factories.numpy_minimum_eigensolver_factory import NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.second_q.algorithms.excited_states_solvers.eigensolver_factories.numpy_eigensolver_factory import NumPyEigensolverFactory
from qiskit_nature.second_q.algorithms.excited_states_solvers import QEOM, ExcitedStatesEigensolver

#Runtime
from qiskit_ibm_runtime import (QiskitRuntimeService, Session,
                                Estimator as RuntimeEstimator)
from qiskit_ibm_runtime.options import Options, ResilienceOptions, SimulatorOptions, TranspilationOptions, ExecutionOptions

#PySCF
from functools import reduce
import scipy.linalg
from pyscf import scf
from pyscf import gto, dft
from pyscf import mcscf, fci
from functools import reduce
from pyscf.mcscf import avas, dmet_cas

#Python odds and ends
import matplotlib
import matplotlib.pyplot as plt
import pylab
import numpy as np
import os
import pyscf
from IPython.display import display, clear_output
import mapomatic as mm

from datetime import datetime
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

IBMQ.load_account()
provider = IBMQ.get_provider(group='deployed')
service = QiskitRuntimeService(channel='ibm_quantum')
#set Backends
#simulators
backend_stv = Aer.get_backend('aer_simulator_statevector')
#Real Devices
backend_nair= provider.get_backend('ibm_nairobi')
backend_manil = provider.get_backend('ibmq_manila')
backend_qsm_ibm=provider.get_backend('ibmq_qasm_simulator')
seed=42

#solvers
npme = NumPyMinimumEigensolver()
npe=NumPyEigensolver()

/var/folders/ts/54g0mcnn4glc0n7qph2hqb2c0000gn/T/ipykernel_76886/4269044919.py:61: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.load_account()
/var/folders/ts/54g0mcnn4glc0n7qph2hqb2c0000gn/T/ipykernel_76886/4269044919.py:61: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


Helper functions to save and read results

In [2]:
## Python program to store list to file using pickle module
import pickle

# write list to binary file
def write_list(a_list,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
        
def write_dict(a_dict,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(a_dict, fp,protocol=pickle.HIGHEST_PROTOCOL)
        print('Done writing dict into a binary file')

# Read list to memory
def read(filename):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

Optional Custom VQE class

# HW Results

Let's start with smallest system first

## Mg2+ + H2O

We begin by looking at the Session IDs for the Hardware jobs

Montreal:

cfvn06vb5bed9m42ph60 - 3.0 Angstroms

cfvn074akfev56pgpilg - 3.3 Angstroms

cfvn0anb5bed9m42pjg0 - 0.3 Angstroms

cfvn0cjptln070a684rg - 2.1 Angstroms

cfvn0djptln070a685j0 - 0.6 Angstroms

cfvn0df18ijt3i1jbk80 - 1.5 Angstroms

cfvn0ffb5bed9m42pn60 - 0.9 Angstroms

cfvn0fjptln070a68770 - 1.8 Angstroms

cfvn0frptln070a687a0 - 1.2 Angstroms

cfvn0j3ptln070a689o0 - 2.4 Angstroms

cfvn0lgi2e9ud6sb1tng - 2.7 Angstroms

Taking those session IDS to the IBM Quantum platform we can grab the last jobs for each session.

In [ ]:
#Session ID - Job ID

cfvn06vb5bed9m42ph60 - cfvqi9oi2e9ud6sdq5d0 - 3.0
cfvn074akfev56pgpilg - cfvs7roi2e9ud6seuoo0 - 3.3
cfvn0anb5bed9m42pjg0 - cg00ek3ptln070acaetg - 0.3
cfvn0cjptln070a684rg - cg07b46tm3os8d3ua0l0 - 2.1
cfvn0djptln070a685j0 - cg0lebasm4lj6dkss870 - 0.6
cfvn0df18ijt3i1jbk80 - cg24qrpg15ojgku47t40 - 1.5
cfvn0ffb5bed9m42pn60 - cg29eg9g15ojgku7qho0 - 0.9
cfvn0fjptln070a68770 - cg2afnt2h484fadafcu0 - 1.8
cfvn0frptln070a687a0 - cg2bct2sm4lk6m6ij9i0 - 1.2
cfvn0j3ptln070a689o0 - cg2dgsljvmq2dg2jgn2g - 2.4
cfvn0lgi2e9ud6sb1tng - cg2i8n1ks25cvuap8ctg - 2.7


We now retrieve those job results and order them

In [187]:
service=QiskitRuntimeService()
jobs=[]
jobs.append(service.job('cg00ek3ptln070acaetg'))
jobs.append(service.job('cg0lebasm4lj6dkss870')) 
jobs.append(service.job('cg29eg9g15ojgku7qho0'))
jobs.append(service.job('cg2bct2sm4lk6m6ij9i0'))
jobs.append(service.job('cg24qrpg15ojgku47t40'))
jobs.append(service.job('cg2afnt2h484fadafcu0'))
jobs.append(service.job('cg07b46tm3os8d3ua0l0'))
jobs.append(service.job('cg2dgsljvmq2dg2jgn2g'))
jobs.append(service.job('cg2i8n1ks25cvuap8ctg'))
jobs.append(service.job('cfvqi9oi2e9ud6sdq5d0'))
jobs.append(service.job('cfvs7roi2e9ud6seuoo0'))

We can extract the VQE computed part like so:

In [188]:
values=[]
distances=np.arange(0.3, 3.5, 0.3)
for i in range(len(distances)):
    values.append(jobs[i].result().values[0])
values

[-15.080071186302474,
 -16.149779369049785,
 -16.991558674690737,
 -17.261040659229266,
 -16.07687064438014,
 -15.994346240530488,
 -15.749219532346872,
 -15.130543032226084,
 -14.926419222561403,
 -14.885587551144582,
 -14.239867656427263]

In [189]:
class_comp

[-15.862766228556374,
 -17.312477071937746,
 -17.522080419326386,
 -17.59841082885856,
 -17.14393411077303,
 -16.62718362134687,
 -16.18671581493006,
 -15.832532865916342,
 -15.5501268441866,
 -15.32295945741001]

In [193]:
sum=[]
for i in range(len(class_comp)):
    sum.append(values[i]-class_comp[i])
sum

[0.7826950422539003,
 1.1626977028879608,
 0.5305217446356494,
 0.33737016962929545,
 1.0670634663928915,
 0.6328373808163814,
 0.4374962825831883,
 0.7019898336902575,
 0.6237076216251971,
 0.43737190626542777]

Now we populate the VQEResult()

In [191]:
from qiskit.algorithms.minimum_eigensolvers import VQEResult
results=[]
for i in range(len(distances)):
    result=VQEResult()
    result.eigenvalue = jobs[i].result().values[0]
    result.optimal_value = jobs[i].result().values[0]
    results.append(result)


In [69]:
distances

array([0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3. , 3.3])

In [71]:
print(results[0])

{   'aux_operators_evaluated': None,
    'cost_function_evals': None,
    'eigenvalue': -14.885587551144582,
    'optimal_circuit': None,
    'optimal_parameters': None,
    'optimal_point': None,
    'optimal_value': -14.885587551144582,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': None}


We need to interpret that result, for that we need to recreate the Electronic Structure problems

In [99]:
def make_driver(d):
    
    
    molecule = MoleculeInfo(
             # coordinates in Angstrom
                     symbols=['O','H','H','Mg'],
                     coords=[
                            # (d+0.504284,0.0,0.758602),
                            # (d,0.0,0.0),
                            # (d+2*0.504284,0.0,0.0),
                            # (0.0, 0.0, 0.0),
                            (0.0,0.0,0.0),
                            (-0.504284,0.0,-0.758602),
                            (0.504284,0.0,-0.758602),
                            (0.0, 0.0, d),
                            ],
                     multiplicity=1,  # = 2*spin + 1
                     charge=2,
                     units=DistanceUnit.ANGSTROM
                    )
    
    #Set driver
    #driver = PySCFDriver.from_molecule(molecule, basis="sto3g", method=MethodType.ROHF)
    #driver.xc_functional='pbe,pbe'
    driver = PySCFDriver.from_molecule(molecule, basis="6-31g*", method=MethodType.ROKS)
    driver.xc_functional='b3lyp'
    driver.conv_tol = 1e-6

    #Get properties
    problem = driver.run()
    

    return driver, problem

def make_qubit_op(d,og_problem, mapper,freeze_core):
    mol = gto.Mole()
    mol.atom = [
        # ['O',(d+0.504284,0.0,0.758602)],
        # ['H',(d,0.0,0.0),],
        # ['H',(d+2*0.504284,0.0,0.0)],
        # ['Mg',(0.0, 0.0, 0.0)]
        ['O',(0.0,0.0,0)],
        ['H',(-0.504284,0.0,-0.758602),],
        ['H',(0.504284,0.0,-0.758602)],
        ['Mg',(0.0, 0.0, d)]
        ]
    mol.charge=2
    mol.basis = '6-31g*'
    mol.spin = 0
    mol.build()
    
    #mf= scf.ROHF(mol).x2c()
    mf = dft.ROKS(mol).density_fit(auxbasis='def2-universal-jfit')
    mf.xc ='pbe,pbe'
    mf.max_cycle = 50
    mf.conv_tol = 1e-6
    
    first_run=mf.kernel()
    a = mf.stability()[0]
    if(mf.converged):
        energy=first_run
    else:
        mf.max_cycle = 80
        mf.conv_tol = 1e-6
        mf = scf.newton(mf)
        scnd_run=mf.kernel(dm0 = mf.make_rdm1(a,mf.mo_occ)) # using rdm1 constructed from stability analysis
      #mf.kernel(mf.make_rdm1()) #using the rdm from the non-converged calculation
        if(mf.converged):
            energy=scnd_run
        else:
            mf.conv_tol = 1e-6
            mf.max_cycle = 80
            mf = scf.newton(mf) #Second order solver
            energy=mf.kernel(dm0 = mf.make_rdm1())


    ao_labels = ['Mg 2p', 'O 2p']
    avas_obj = avas.AVAS(mf, ao_labels)
    avas_obj.kernel()
    weights=np.append(avas_obj.occ_weights,avas_obj.vir_weights)
    weights=(weights>0.2)*weights
    orbs=np.nonzero(weights)
    orbs=np.nonzero(weights)
    
    # transformer = ActiveSpaceTransformer(
    #         num_electrons=(int(avas_obj.nelecas/2),int(avas_obj.nelecas/2)), #Electrons in active space
    #         num_spatial_orbitals=avas_obj.ncas+1, #Orbitals in active space
    #         active_orbitals=orbs[0].tolist().append(orbs[0][-1]+1)
    #     )
    transformer = ActiveSpaceTransformer(
            num_electrons=(3,3), #Electrons in active space
            num_spatial_orbitals=4, #Orbitals in active space
            #active_orbitals=orbs[0].tolist().append(orbs[0][-1]+1)
        )
    fz_transformer=FreezeCoreTransformer(freeze_core=freeze_core)
    
    #Define the problem

    problem=transformer.transform(og_problem)
    if freeze_core==True:
        problem=fz_transformer.transform(problem)
        converter = QubitConverter(mapper)
    else:
        converter = QubitConverter(mapper,two_qubit_reduction=True, z2symmetry_reduction='auto')

    hamiltonian=problem.hamiltonian
    second_q_op = hamiltonian.second_q_op()
    
    num_spatial_orbitals = problem.num_spatial_orbitals
    num_particles = problem.num_particles
    
    qubit_op = converter.convert(second_q_op,num_particles=num_particles,sector_locator=problem.symmetry_sector_locator)
    
        

    
    return qubit_op, problem,  converter, energy
    

In [44]:
mapper=ParityMapper()
problems=[]
for dist in distances:
    #Driver
    driver,og_problem=make_driver(dist)
    #Qubit_Op
    qubit_op, problem, converter,hf_energy = make_qubit_op(dist,og_problem,mapper,freeze_core=False)
    problems.append(problem)

converged SCF energy = -233.036314605562
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -268.658419761889
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -273.531032202168
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -275.014052803726
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -275.40265068599
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -275.478859766864
<class 'pyscf.df.df_jk.density_fit.<locals>.DensityFitting'> wavefunction is stable in the internal stability analysis
converged SCF energy = -275.47762165946
<

We can now Interpet results:

In [105]:
print(problem.interpret(results[0]))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -300.888741570929
  - computed part:      -14.239867656427
  - ActiveSpaceTransformer extracted energy part: -286.648873914502
~ Nuclear repulsion energy (Hartree): 26.825396280283
> Total ground state energy (Hartree): -274.063345290646
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  78.76906855]
 


In [192]:
interp_results=[]
for i in range(len(distances)):
    interp_results.append(problems[i].interpret(results[i]))
write_list(interp_results,"MG+H2O/VQE_results/interpreted_results")

Done writing list into a binary file


In [151]:
class_comp=[]
distances=np.arange(0.3,3.3,0.3)
for i in range(len(distances)):
    class_comp.append(classic_results[i].computed_energies[0])

In [139]:
print(problems[9].interpret(results[9]))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -304.682278384906
  - computed part:      -15.080071186302
  - ActiveSpaceTransformer extracted energy part: -289.602207198604
~ Nuclear repulsion energy (Hartree): 30.102122389448
> Total ground state energy (Hartree): -274.580155995458
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  65.16304045]
 


In [152]:
class_comp

[-15.862766228556374,
 -17.312477071937746,
 -17.522080419326386,
 -17.59841082885856,
 -17.14393411077303,
 -16.62718362134687,
 -16.18671581493006,
 -15.832532865916342,
 -15.5501268441866,
 -15.32295945741001]